本腳本旨在針對113年事故傷亡結果做簡易整🈴後給出一份描述資料分佈的純文字檔.txt，以利團隊後續更快速掌握資料工程子任務拆分。

根據團隊先前的操作經驗，發現pandas.read_csv()的dtype參數有兩種設定邏輯：

(1) 直接在dtype字典中說明各欄位的資料進入SQL系統時的資料型態(import sqlalchemy)。
(2) 先以pandas擁有的資料型別，傳入dtype(無需import sqlalchemy)。
(3) 於本jupyter notebook測試時，發現方法(1)會疑似出現解讀不了的問題。例如：TypeError: Cannot interpret 'VARCHAR(length=50)' as a data type。 <-- 問題點尚需釐清。

1. 引入需要的模組

In [2]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, types, text # 用以引入寫進資料庫時

2. 讀取A1&A2車禍事件紀錄表(csv)

In [ ]:
# # 讀檔前先定義資料型別，寫法一，在讀取csv時強行轉成sql的資料型態：
# col_map = {"發生年度": {"name": "accident_year", "type": types.VARCHAR(50)},
#     "發生月份": {"name": "accident_month", "type": types.VARCHAR(50)},
#     "發生日期": {"name": "accident_date", "type": types.VARCHAR(50)},
#     "發生時間": {"name": "accident_time", "type": types.VARCHAR(50)},
#     "事故類別名稱": {"name": "accident_category", "type": types.VARCHAR(50)},
#     "處理單位名稱警局層": {"name": "police_department", "type": types.VARCHAR(100)},
#     "發生地點": {"name": "accident_location", "type": types.VARCHAR(150)},
#     "天候名稱": {"name": "weather_condition", "type": types.VARCHAR(20)},
#     "光線名稱": {"name": "light_condition", "type": types.VARCHAR(30)},
#     "道路類別-第1當事者-名稱": {"name": "road_type_primary_party", "type": types.VARCHAR(30)},
#     "速限-第1當事者": {"name": "speed_limit_primary_party", "type": types.SMALLINT},
#     "道路型態大類別名稱": {"name": "road_form_major", "type": types.VARCHAR(30)},
#     "道路型態子類別名稱": {"name": "road_form_minor", "type": types.VARCHAR(30)},
#     "事故位置大類別名稱": {"name": "accident_position_major", "type": types.VARCHAR(30)},
#     "事故位置子類別名稱": {"name": "accident_position_minor", "type": types.VARCHAR(30)},
#     "路面狀況-路面鋪裝名稱": {"name": "road_surface_pavement", "type": types.VARCHAR(30)},
#     "路面狀況-路面狀態名稱": {"name": "road_surface_condition", "type": types.VARCHAR(30)},
#     "路面狀況-路面缺陷名稱": {"name": "road_surface_defect", "type": types.VARCHAR(30)},
#     "道路障礙-障礙物名稱": {"name": "road_obstacle", "type": types.VARCHAR(30)},
#     "道路障礙-視距品質名稱": {"name": "sight_distance_quality", "type": types.VARCHAR(30)},
#     "道路障礙-視距名稱": {"name": "sight_distance", "type": types.VARCHAR(30)},
#     "號誌-號誌種類名稱": {"name": "traffic_signal_type", "type": types.VARCHAR(30)},
#     "號誌-號誌動作名稱": {"name": "traffic_signal_action", "type": types.VARCHAR(30)},
#     "車道劃分設施-分向設施大類別名稱": {"name": "lane_divider_direction_major", "type": types.VARCHAR(40)},
#     "車道劃分設施-分向設施子類別名稱": {"name": "lane_divider_direction_minor", "type": types.VARCHAR(40)},
#     "車道劃分設施-快慢車道間名稱": {"name": "lane_divider_fast_slow", "type": types.VARCHAR(40)},
#     "車道劃分設施-主車道線名稱": {"name": "lane_divider_main_general", "type": types.VARCHAR(40)},
#     "車道劃分設施-路邊邊線名稱": {"name": "lane_edge_marking", "type": types.VARCHAR(40)},
#     "事故類型及型態大類別名稱": {"name": "accident_type_major", "type": types.VARCHAR(40)},
#     "事故類型及型態子類別名稱": {"name": "accident_type_minor", "type": types.VARCHAR(100)},
#     "肇因研判大類別名稱-主要": {"name": "cause_analysis_major_primary", "type": types.VARCHAR(100)},
#     "肇因研判子類別名稱-主要": {"name": "cause_analysis_minor_primary", "type": types.VARCHAR(200)},
#     "死亡受傷人數": {"name": "casualties_count", "type": types.VARCHAR(30)},
#     "當事者順位": {"name": "party_sequence", "type": types.INTEGER},
#     "當事者區分-類別-大類別名稱-車種": {"name": "vehicle_type_major", "type": types.VARCHAR(40)},
#     "當事者區分-類別-子類別名稱-車種": {"name": "vehicle_type_minor", "type": types.VARCHAR(100)},
#     "當事者屬-性-別名稱": {"name": "gender", "type": types.VARCHAR(30)},
#     "當事者事故發生時年齡": {"name": "age", "type": types.SMALLINT},
#     "保護裝備名稱": {"name": "protective_equipment", "type": types.VARCHAR(30)},
#     "行動電話或電腦或其他相類功能裝置名稱": {"name": "mobile_device_usage", "type": types.VARCHAR(30)},
#     "當事者動作大類別名稱": {"name": "party_action_major", "type": types.VARCHAR(40)},
#     "當事者動作子類別名稱": {"name": "party_action_minor", "type": types.VARCHAR(40)},
#     "碰撞部位-最初大類別名稱": {"name": "impact_point_major_initial", "type": types.VARCHAR(40)},
#     "碰撞部位-最初子類別名稱": {"name": "impact_point_minor_initial", "type": types.VARCHAR(40)},
#     "碰撞部位-其他大類別名稱": {"name": "impact_point_major_other", "type": types.VARCHAR(40)},
#     "碰撞部位-其他子類別名稱": {"name": "impact_point_minor_other", "type": types.VARCHAR(40)},
#     "肇因研判大類別名稱-個別": {"name": "cause_analysis_major_individual", "type": types.VARCHAR(40)},
#     "肇因研判子類別名稱-個別": {"name": "cause_analysis_minor_individual", "type": types.VARCHAR(200)},
#     "肇事逃逸類別名稱-是否肇逃": {"name": "hit_and_run", "type": types.VARCHAR(20)},
#     "經度": {"name": "longitude", "type": types.DECIMAL(10, 6)},
#     "緯度": {"name": "latitude", "type": types.DECIMAL(10, 6)}
# }
# print(len(col_map))

In [22]:
# 讀檔前先定義資料型別，寫法二，在讀取csv時遵循pandas自己的資料型態：
col_map = {"發生年度": {"name": "accident_year", "type": int},
    "發生月份": {"name": "accident_month", "type": int},
    "發生日期": {"name": "accident_date", "type": object},
    "發生時間": {"name": "accident_time", "type": object},
    "事故類別名稱": {"name": "accident_category", "type": object},
    "處理單位名稱警局層": {"name": "police_department", "type": object},
    "發生地點": {"name": "accident_location", "type": object},
    "天候名稱": {"name": "weather_condition", "type": object},
    "光線名稱": {"name": "light_condition", "type": object},
    "道路類別-第1當事者-名稱": {"name": "road_type_primary_party", "type": object},
    "速限-第1當事者": {"name": "speed_limit_primary_party", "type": object},
    "道路型態大類別名稱": {"name": "road_form_major", "type": object},
    "道路型態子類別名稱": {"name": "road_form_minor", "type": object},
    "事故位置大類別名稱": {"name": "accident_position_major", "type": object},
    "事故位置子類別名稱": {"name": "accident_position_minor", "type": object},
    "路面狀況-路面鋪裝名稱": {"name": "road_surface_pavement", "type": object},
    "路面狀況-路面狀態名稱": {"name": "road_surface_condition", "type": object},
    "路面狀況-路面缺陷名稱": {"name": "road_surface_defect", "type": object},
    "道路障礙-障礙物名稱": {"name": "road_obstacle", "type": object},
    "道路障礙-視距品質名稱": {"name": "sight_distance_quality", "type": object},
    "道路障礙-視距名稱":  {"name": "sight_distance", "type": object},
    "號誌-號誌種類名稱": {"name": "traffic_signal_type", "type": object},
    "號誌-號誌動作名稱": {"name": "traffic_signal_action", "type": object},
    "車道劃分設施-分向設施大類別名稱": {"name": "lane_divider_direction_major", "type": object},
    "車道劃分設施-分向設施子類別名稱": {"name": "lane_divider_direction_minor", "type": object},
    "車道劃分設施-快慢車道間名稱": {"name": "lane_divider_fast_slow", "type": object},
    "車道劃分設施-主車道線名稱": {"name": "lane_divider_main_general", "type": object},
    "車道劃分設施-路邊邊線名稱": {"name": "lane_edge_marking", "type": object},
    "事故類型及型態大類別名稱": {"name": "accident_type_major", "type": object},
    "事故類型及型態子類別名稱": {"name": "accident_type_minor", "type": object},
    "肇因研判大類別名稱-主要": {"name": "cause_analysis_major_primary", "type": object},
    "肇因研判子類別名稱-主要": {"name": "cause_analysis_minor_primary", "type": object},
    "死亡受傷人數": {"name": "casualties_count", "type": object},
    "當事者順位": {"name": "party_sequence", "type": int},
    "當事者區分-類別-大類別名稱-車種": {"name": "vehicle_type_major", "type": object},
    "當事者區分-類別-子類別名稱-車種": {"name": "vehicle_type_minor", "type": object},
    "當事者屬-性-別名稱": {"name": "gender", "type": object},
    "當事者事故發生時年齡": {"name": "age", "type": object},
    "保護裝備名稱": {"name": "protective_equipment", "type": object},
    "行動電話或電腦或其他相類功能裝置名稱": {"name": "mobile_device_usage", "type": object},
    "當事者動作大類別名稱": {"name": "party_action_major", "type": object},
    "當事者動作子類別名稱": {"name": "party_action_minor", "type": object},
    "碰撞部位-最初大類別名稱": {"name": "impact_point_major_initial", "type": object},
    "碰撞部位-最初子類別名稱": {"name": "impact_point_minor_initial", "type": object},
    "碰撞部位-其他大類別名稱": {"name": "impact_point_major_other", "type": object},
    "碰撞部位-其他子類別名稱": {"name": "impact_point_minor_other", "type": object},
    "肇因研判大類別名稱-個別": {"name": "cause_analysis_major_individual", "type": object},
    "肇因研判子類別名稱-個別": {"name": "cause_analysis_minor_individual", "type": object},
    "肇事逃逸類別名稱-是否肇逃": {"name": "hit_and_run", "type": object},
    "經度": {"name": "longitude", "type": float},
    "緯度": {"name": "latitude", "type": float},
}
# print(len(col_map))

In [24]:
def read_and_concat_csv(file_dir: Path, keyword_of_file_name: str, dp: dict) -> pd.DataFrame:
    """Read the files about the A1 or A2 accidents in the 'file_dir' folder and then
    concat, load into a new DataFrame.Keyword_of_file_name: A1/A2"""

    all_df = []
    target_file_name = f'*{keyword_of_file_name}*.csv'
    count = 0
    # 用glob遍歷來源資料夾，然後逐一讀取csv直至存進all_df list暫存。
    for a_csv in file_dir.glob(target_file_name):
        print('正在讀取:', a_csv.name)
        count += 1
        # 每個csv檔的尾巴有兩筆無法正常按照dtype轉換，故跳過這2列 #skipfooter只有engine設為python時才有支援。
        df = pd.read_csv(a_csv, dtype=dp,
                         skipfooter=2, engine='python')  
        df = df.iloc[:-2]
        df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

        # 清理日期欄位的格式   
        df['發生日期'] = pd.to_datetime(df['發生日期']) #20260101這種格式pandas自己可以識別清楚且保證與SQL的日期格式相容。

        # 清理時間欄位的格式
        df['發生時間'] = pd.to_datetime(df['發生時間'], format='%H%M%S').dt.time # 雷點：不支援有冒號的寫法%HH:%MM:%SS。也不支援%HH%MM%SS

        # 將欄位名稱從中文轉成英文
        col_name = [v['name'] for v in col_map.values()]
        df.columns = col_name
        
        # 將每一個df物件暫存到list中
        all_df.append(df)
    
    # 遍歷all_df中的df，逐一串接。
    final_df = pd.concat(all_df)
    final_df = final_df.drop_duplicates() # 去除重複項if exist

    # 將經緯度合併後串接在新的一欄
    final_df["long, lat"] = final_df["longitude"].astype(
        str) + ', ' + final_df["latitude"].astype(str)
    
    print(f"總共處理了{count}個csv檔.")
    final_df.head()
    return final_df

產出閱讀報告，描述每一種欄位所具備的值域多廣、每種值有多少筆

In [25]:
def annual_report_at_fault_driver_only(df_a1: pd.DataFrame, df_a2: pd.DataFrame, save_file_path: Path):
    """Open the gotten DataFrames and then filtered the the drivers that were
    not at-fault drivers."""

    with save_file_path.open(mode="w", encoding='utf-8') as f:
        content = []
        for col in df_a1.columns:
            data_a1 = df_a1[df_a1['party_sequence'] == 1][col]
            data_a2 = df_a2[df_a2['party_sequence'] == 1][col]
            values_count_a1 = data_a1.nunique()
            values_a1 = data_a1.value_counts()
            values_count_a2 = data_a2.nunique()
            values_a2 = data_a2.value_counts()

            abstract_a1 = f'Column Name「{col}」當中，A1類別總共{values_count_a1}種值, 分別為: '
            values_a1['total'] = data_a1.count()
            content.append(abstract_a1)
            content.append(str(values_a1))

            abstract_a2 = f'\n而A2類別總共{values_count_a2}種值, 分別為: '
            values_a2['total'] = data_a2.count()
            content.append(abstract_a2)
            content.append(str(values_a2))
            content.append('--------------------------------------')

        f.write('\n'.join(content))


In [ ]:
if __name__ == "__main__":
    # curr_dir = Path(__file__).resolve().parent # 在Jupyter Notebook 中， __file__  變數不存在，因為 Notebook 不是像  .py  腳本那樣從檔案載入執行，而是透過互動式 kernel 逐 cell 運行，所以會拋出  NameError 。同理，也不會有__NAME__這個變數。
    curr_dir = Path.cwd().resolve().parent # curr_dir就是01_Nightmarket....這層
    src_dir = curr_dir/"00_data_sources/113年傷亡道路交通事故資料"

    dtypes = {k: v['type'] for k, v in col_map.items()}
    col_name = [v['name'] for v in col_map.values()]
    final_df_A1 = read_and_concat_csv(src_dir, keyword_of_file_name='A1', dp=dtypes)
    final_df_A2 = read_and_concat_csv(src_dir, keyword_of_file_name='A2', dp=dtypes)
    annual_report_at_fault_driver_only(final_df_A1, final_df_A2, save_file_path=curr_dir/"03_output/A1&A2_log_comparison_at-fault-driver_only.txt")

正在讀取: 113年度A1交通事故資料.csv
總共處理了1個csv檔.
正在讀取: 113年度A2交通事故資料_3.csv
正在讀取: 113年度A2交通事故資料_2.csv
正在讀取: 113年度A2交通事故資料_12.csv
正在讀取: 113年度A2交通事故資料_10.csv
正在讀取: 113年度A2交通事故資料_1.csv
正在讀取: 113年度A2交通事故資料_11.csv
正在讀取: 113年度A2交通事故資料_5.csv
正在讀取: 113年度A2交通事故資料_4.csv
正在讀取: 113年度A2交通事故資料_6.csv
正在讀取: 113年度A2交通事故資料_7.csv
正在讀取: 113年度A2交通事故資料_9.csv
正在讀取: 113年度A2交通事故資料_8.csv
總共處理了12個csv檔.


In [28]:
# 也把final_df_A1與final_df_A2輸出成csv，以進行後續二次清理 <- 這裡有點困惑，如果不需要清，是不是可以把e-這份直接跑到l-?也就是直接存入資料庫。

final_df_A1.to_csv(curr_dir/"02_processed_data/A1_summary_113.csv", index= False)
final_df_A2.to_csv(curr_dir/"02_processed_data/A2_summary_113.csv", index= False)